# PoC to extract paragraphs from lines

In [14]:
# pip stuff needed
#%pip install openpyxl
#%pip install nltk
#%pip install spacy


# Wellbore_nema 
# From release date extract year 

                                              0.0/12.1 MB ? eta -:--:--
                                              0.1/12.1 MB 2.2 MB/s eta 0:00:06
     -                                        0.5/12.1 MB 4.7 MB/s eta 0:00:03
     --                                       0.8/12.1 MB 6.5 MB/s eta 0:00:02
     ----                                     1.3/12.1 MB 7.6 MB/s eta 0:00:02
     ------                                   2.0/12.1 MB 8.5 MB/s eta 0:00:02
     --------                                 2.7/12.1 MB 9.4 MB/s eta 0:00:01
     -----------                              3.4/12.1 MB 10.5 MB/s eta 0:00:01
     --------------                           4.3/12.1 MB 11.4 MB/s eta 0:00:01
     ---------------                          4.8/12.1 MB 11.3 MB/s eta 0:00:01
     ------------------                       5.5/12.1 MB 11.8 MB/s eta 0:00:01
     ---------------------                    6.5/12.1 MB 12.5 MB/s eta 0:00:01
     -------------------------                7.7/12.1


[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [16]:
# Imports
import pandas as pd
import re
import nltk
import spacy
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\aleksander.jakobsen\AppData\Roaming\nltk_data
[nltk_data]     ...
[nltk_data]   Package punkt is already up-to-date!


True

In [19]:
from spacy.cli import download
download('en_core_web_sm')

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [3]:
path_to_file = "../data/Norway - Diskos reports_5.xlsx"
df = pd.read_excel(path_to_file)
columns = df.columns
columns

Index(['_id', 'filename', 'page', 'content', 'possible_language', 'langdetect',
       'content_could_be_natural_language', 'content_scrubbed_light',
       'content_scrubbed_light_could_be_natural_language', 'WELLBORE NAME',
       'WELL NAME', 'TYPE', 'INFO ITEM GROUP TYPE', 'INFO ITEM TYPE', 'TITLE',
       'DESCRIPTION', 'CREATOR BA NAME', 'Short Dataset Id',
       'Required Dataset', 'ROW CREATED DATE', 'PUBLIC', 'PUBLIC ACCESS',
       'RELEASE DATE', 'FILE FORMAT', 'Size', 'DATA ORGANIZATION',
       'DATA COLLECTION', 'CREATION PROCESS', 'DATA DOMAIN', 'REMARK',
       'UNLOAD FILE PATH', 'UNLOAD FILE NAME', 'INFORMATION ITEM ID',
       'Dataset Id', 'Last Modified', 'Update Time'],
      dtype='object')

In [44]:
df

,_id,filename,page,content,possible_language,langdetect,content_could_be_natural_language,content_scrubbed_light,content_scrubbed_light_could_be_natural_language,WELLBORE NAME,...,DATA COLLECTION,CREATION PROCESS,DATA DOMAIN,REMARK,UNLOAD FILE PATH,UNLOAD FILE NAME,INFORMATION ITEM ID,Dataset Id,Last Modified,Update Time
0,997d83feeb81f8044b6447210ab739ad1e9a4f7f,15_6-5__WELL__15-06-05_PB-706-0719.pdf,6,The Upper Dogger The Interval from 3627 to 368...,en,[en:0.9999977940633832],True,The Upper Dogger The Interval from 3627 to 368...,True,15/6-5,...,DISKOS SCANNED COMPLETION REPORTS,INTERP,WL.CMPL,NaN,NaN,NaN,3.182072e+08,14.05.01.00,2022-03-29 00:00:00,2023-08-28 12:22:54
1,997d83feeb81f8044b6447210ab739ad1e9a4f7f,15_6-5__WELL__15-06-05_PB-706-0719.pdf,7,The above is underlain by 272 in thick sandsto...,en,[en:0.9999952423862815],True,The above is underlain by 272 in thick sandsto...,True,15/6-5,...,DISKOS SCANNED COMPLETION REPORTS,INTERP,WL.CMPL,NaN,NaN,NaN,3.182072e+08,14.05.01.00,2022-03-29 00:00:00,2023-08-28 12:22:54
2,997d83feeb81f8044b6447210ab739ad1e9a4f7f,15_6-5__WELL__15-06-05_PB-706-0719.pdf,8,"seen in the main Sleipner reservoir to date, -...",en,[en:0.9999966473882063],True,"seen in the main Sleipner reservoir to date, -...",True,15/6-5,...,DISKOS SCANNED COMPLETION REPORTS,INTERP,WL.CMPL,NaN,NaN,NaN,3.182072e+08,14.05.01.00,2022-03-29 00:00:00,2023-08-28 12:22:54
3,997d83feeb81f8044b6447210ab739ad1e9a4f7f,15_6-5__WELL__15-06-05_PB-706-0719.pdf,9,TABLE WIRELINE LOGGING SUMMARY Well 15/6-5 Log...,en,[en:0.999994410115181],False,TABLE WIRELINE LOGGING SUMMARY Well 15/6-5 Log...,False,15/6-5,...,DISKOS SCANNED COMPLETION REPORTS,INTERP,WL.CMPL,NaN,NaN,NaN,3.182072e+08,14.05.01.00,2022-03-29 00:00:00,2023-08-28 12:22:54
4,997d83feeb81f8044b6447210ab739ad1e9a4f7f,15_6-5__WELL__15-06-05_PB-706-0719.pdf,10,WELLSITE SAMPLE DESCRIPTION HOLE SIZE: GEOL: R...,en,[en:0.9999961362929385],True,WELLSITE SAMPLE DESCRIPTION HOLE SIZE: GEOL: R...,True,15/6-5,...,DISKOS SCANNED COMPLETION REPORTS,INTERP,WL.CMPL,NaN,NaN,NaN,3.182072e+08,14.05.01.00,2022-03-29 00:00:00,2023-08-28 12:22:54
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
252376,d8b5ec0eb354bbf9e4865565faa4a890dbabfb45,15_9-12__ROCK AND CORE__CORE_SCAL_1982-10-30_R...,48,"wycousawNmp,moom-Liwm e8-1301 (opds aJod luaoJ...",en,"[en:0.5714274116776232, pt:0.2857141457066634,...",False,"wycousawNmp,moom-Liwm e8-1301 (opds aJod luaoJ...",False,15/9-12,...,DISKOS LEGACY DATA,LAB,WL.RC.SCAL,NaN,WELL_15_9-12/WB_309-L/CORE_DATA_1982-10-30,CORE_SCAL_1982-10-30_REPORT_1.PDF,1.017913e+10,02.25.01.00,2023-04-25 00:00:00,2023-07-12 07:25:22
252377,d8b5ec0eb354bbf9e4865565faa4a890dbabfb45,15_9-12__ROCK AND CORE__CORE_SCAL_1982-10-30_R...,49,"IrVEN,114111NMPuelgeoW-liO'su af-EM11 (Boods a...",en,"[pt:0.8571396095099522, en:0.1428589548695462]",False,"IrVEN,114111NMPuelgeoW-liO'su af-EM11 (Boods a...",False,15/9-12,...,DISKOS LEGACY DATA,LAB,WL.RC.SCAL,NaN,WELL_15_9-12/WB_309-L/CORE_DATA_1982-10-30,CORE_SCAL_1982-10-30_REPORT_1.PDF,1.017913e+10,02.25.01.00,2023-04-25 00:00:00,2023-07-12 07:25:22
252378,d8b5ec0eb354bbf9e4865565faa4a890dbabfb45,15_9-12__ROCK AND CORE__CORE_SCAL_1982-10-30_R...,50,Pressure (bar II Capillary Pressure Curve Comp...,en,"[en:0.7142842662068939, pt:0.28571429811363974]",False,Pressure (bar II Capillary Pressure Curve Comp...,False,15/9-12,...,DISKOS LEGACY DATA,LAB,WL.RC.SCAL,NaN,WELL_15_9-12/WB_309-L/CORE_DATA_1982-10-30,CORE_SCAL_1982-10-30_REPORT_1.PDF,1.017913e+10,02.25.01.00,2023-04-25 00:00:00,2023-07-12 07:25:22
252379,d8b5ec0eb354bbf9e4865565faa4a890dbabfb45,15_9-12__ROCK AND CORE__CORE_SCAL_1982-10-30_R...,51,Company STATQIL A/S Depth.. S.Q57...05...........,en,"[en:0.571427361092836, pt:0.42857077727642967]",False,Company STATQIL A/S Depth.. S.Q57...05Gr. Dens...,False,15/9-12,...,DISKOS LEGACY DATA,LAB,WL.RC.SCAL,NaN,WELL_15_9-12/WB_309-L/CORE_DATA_1982-10-30,CORE_SCAL_1982-10-30_REPORT_1.PDF,1.017913e+10,02.25

In [16]:
df.dtypes

_id                                                  object
filename                                             object
page                                                  int64
content                                              object
possible_language                                    object
langdetect                                           object
content_could_be_natural_language                      bool
content_scrubbed_light                               object
content_scrubbed_light_could_be_natural_language       bool
WELLBORE NAME                                        object
WELL NAME                                            object
TYPE                                                 object
INFO ITEM GROUP TYPE                                 object
INFO ITEM TYPE                                       object
TITLE                                                object
DESCRIPTION                                          object
CREATOR BA NAME                         

Filters:
- "INFO ITEM TYPE" on "DISCOVERY_EVALUATION_REPORT"
- "content_could_be_natural_language" = True
- "possible_language" = "en"



In [23]:
    def concatonate_docs(df):
        """
        Concatenates lines with same _id
        params: df (DataFrame) - The DataFrame to process
        returns: DataFrame with concatenated content by '_id'
        """
        grouped_content = df.groupby('_id')['content'].apply(lambda x: ''.join(x.astype(str)))
        result_df = grouped_content.reset_index(name='concatenated_content')
        return result_df





    # Load spaCy's pre-trained English model. This step needs to be done only once,
    # usually at the beginning of your script. The model is used for sentence tokenization.
    nlp = spacy.load("en_core_web_sm")

    def filter_sentences(doc):
        """
        Filters sentences to exclude document headings or enumerations.
        
        Params:
        - doc (spaCy Doc): The spaCy Doc object containing the processed text.
        
        Returns:
        - list: A list of sentences after excluding non-standard text patterns.
        """
        sentences = []
        for sent in doc.sents:
            # Matches sequences like "1.", "1.1", "2.3.4", etc., followed by a space and a capital letter
            pattern = r"\d+(\.\d+)*\s+[A-Z]"
            
            # Check if the sentence does not start with the pattern (to filter out titles/headings)
            if not re.match(pattern, sent.text):
                sentences.append(sent.text.strip())  # Add the sentence to the list after stripping whitespace
        return sentences

    def define_passages(df, global_passage_id, passage_length):
        """
        Splits documents into passages of complete sentences based on the specified passage length.
        
        Params:
        - df (DataFrame): DataFrame with documents concatenated by '_id'.
        - global_passage_id (int): An integer ID that uniquely identifies each passage.
        - passage_length (int): The number of sentences in each passage.
        
        Returns:
        - DataFrame: A new DataFrame with columns '_id', 'Passage_id', 'passage'.
        - int: The updated global passage ID after processing the DataFrame.
        """
        # Initialize a list to store passage data
        passages_data = []

        # Iterate through each row in the DataFrame
        for index, row in df.iterrows():
            # Extract the document ID and concatenated content for the current row
            doc_id = row['_id']
            content = row['concatenated_content']
            
            # Process the content using spaCy to tokenize into sentences
            doc = nlp(content)
            
            # Filter the sentences using the custom filter function
            sentences = filter_sentences(doc)

            # Group sentences into passages of the specified length
            for i in range(0, len(sentences), passage_length):
                # Join sentences to form a single passage text
                passage = ' '.join(sentences[i:i+passage_length])
                # Append the passage data to the list, including the unique passage ID
                passages_data.append([doc_id, global_passage_id, passage])
                # Increment the unique passage ID for the next passage
                global_passage_id += 1

        # Create a DataFrame from the passage data
        passages_df = pd.DataFrame(passages_data, columns=['_id', 'Passage_id', 'passage'])
        
        # Return the DataFrame and the updated passage ID
        return passages_df, global_passage_id


    def base_df(df, passage_length):
        """
        Process the DataFrame to return passages for all unique "INFO ITEM TYPE"

        Params:
        - df (DataFrame): Original DataFrame with different "INFO ITEM TYPE"s
        - passage_length (int): Number of sentences that make up a passage

        Returns:
        - DataFrame with "INFO_ITEM_TYPE", "_id", "passage_id", "passage"
        """
        global_passage_id = 0
        result_dfs = []  # List to hold intermediate DataFrames

        # Get unique 'INFO ITEM TYPE'
        info_item_types = df['INFO ITEM TYPE'].unique()

        # Process each unique 'INFO ITEM TYPE'
        for item_type in info_item_types:
            # Subset the dataframe by 'INFO ITEM TYPE'
            df_filtered = df[(df['INFO ITEM TYPE'] == item_type) &
                            (df['content_could_be_natural_language'] == True) &
                            (df['possible_language'] == 'en')]\
                            .reset_index(drop=True)
            
            concat_df = concatonate_docs(df_filtered)
            passage_df, global_passage_id = define_passages(concat_df, global_passage_id, passage_length)
            
            # Add back the 'INFO ITEM TYPE' to be part of the result DataFrame
            passage_df['INFO_ITEM_TYPE'] = item_type
            result_dfs.append(passage_df)

        # Concatenate all intermediate DataFrames
        final_df = pd.concat(result_dfs).reset_index(drop=True)
        
        # Reorder columns to match the desired format
        final_df = final_df[['INFO_ITEM_TYPE', '_id', 'Passage_id', 'passage']]

        return final_df

In [42]:
desired_df = base_df(df[df['INFO ITEM TYPE'] == 'DISCOVERY_EVALUATION_REPORT'],passage_length=50) 


In [43]:
desired_df.to_csv("passages_discovery_evaluation_report_50_sentences.csv",index=False)

In [25]:
# Assuming `df` is the DataFrame obtained from the base_df function
disc_eval_df  = desired_df[desired_df['INFO_ITEM_TYPE'] == 'DISCOVERY_EVALUATION_REPORT'])




,INFO_ITEM_TYPE,_id,Passage_id,passage
130481,DISCOVERY_EVALUATION_REPORT,59d3b7de417e13085ff1dbee2ce826ff2fe23165,130481,V FORMATION TESTER S.P.E. PL N5 Ended O.O.H.F....
130482,DISCOVERY_EVALUATION_REPORT,59d3b7de417e13085ff1dbee2ce826ff2fe23165,130482,14 mn Sampling chamber pressure reading at sur...
130483,DISCOVERY_EVALUATION_REPORT,59d3b7de417e13085ff1dbee2ce826ff2fe23165,130483,trip ? Reason for testing: Oil sample for PVT ...
130484,DISCOVERY_EVALUATION_REPORT,601b0da72cc6a2ec5015e66636cd167a4d3802fb,130484,LIST OF CONTENT Abstract I. Introduction 2. Ar...
130485,DISCOVERY_EVALUATION_REPORT,601b0da72cc6a2ec5015e66636cd167a4d3802fb,130485,Appendix C: Core listing + core log. Appendix ...
...,...,...,...,...
130703,DISCOVERY_EVALUATION_REPORT,fc177ae8d5cf8ff470803445387f91e294ee0df7,130703,Analysts Name G. W. -COOPERasp lea alio Immo a...
130704,DISCOVERY_EVALUATION_REPORT,fc177ae8d5cf8ff470803445387f91e294ee0df7,130704,- ' - . - . . - . Average Range Uncorr for ...
130705,DISCOVERY_EVALUATION_REPORT,fc177ae8d5cf8ff470803445387f91e294ee0df7,130705,-... - . . . iIMO II= OM NM MI 11111 MI Mill M...
130706,DISCOVERY_EVALUATION_REPORT,fc177ae8d5cf8ff470803445387f91e294ee0df7,130706,2C 4814m 28.6.76 08.00-12.30 - - 10 litres mud...


In [6]:
def concatonate_docs(df):
    """
    Concatonates lines with same _id
    params:
    returns:
    """
    grouped_content = df.groupby('_id')['content'].apply(lambda x: ''.join(x.astype(str)))
    # Reset the index to turn the Series back into a DataFrame
    result_df = grouped_content.reset_index(name='concatenated_content')
    return result_df

def define_passages(df, global_passage_id, passage_length):
    """
    Splits a document into passages of sentences.
    
    Expects df on format "_id", "concatenated_content"
    
    Params:
    - df: DataFrame with the documents concatenated by '_id'
    - passage_size: The number of sentences in each passage (default is 10)

    Returns:
    - DataFrame with the columns "_id", "concatenated_content", "Passage_id", "passage"
    """
    # Initialize the tokenizer
    tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

    # Prepare a list to store the passages data
    passages_data = []


    # Iterate through each document
    for index, row in df.iterrows():
        doc_id = row['_id']
        content = row['concatenated_content']
        
        # Tokenize the document into sentences
        sentences = tokenizer.tokenize(content)

        # Split sentences into passages
        for i in range(0, len(sentences), passage_size):
            passage = ' '.join(sentences[i:i+passage_size])
            passages_data.append([doc_id, global_passage_id, passage])
            global_passage_id += 1

    # Create a new DataFrame with the passages data
    passages_df = pd.DataFrame(passages_data, columns=['_id', 'Passage_id', 'passage'])

    return passages_df, global_passage_id


def  base_df(df,passage_length):

    #filter input df
    df_filtered =  df[(df['INFO ITEM TYPE'] == 'DISCOVERY_EVALUATION_REPORT') & (df['content_could_be_natural_language'] == True) & (df['possible_language'] == 'en')].reset_index()
    concat_df = concatonate_docs(df_filtered)
    print(concat_df)
    global_passage_id = 0
    passage_df, global_passage_id = define_passages(concat_df,global_passage_id=global_passage_id,passage_length=passage_length)
    return passage_df


In [9]:
concat = concatonate_docs(df)

In [12]:
my_string = concat.loc[1,'concatenated_content']


file_name = "output.txt"

# Open the file in write mode ('w'). This will create the file if it does not exist,
# or overwrite it if it does.
with open(file_name, 'w', encoding='utf-8') as file:
    file.write(my_string)

In [45]:
# def define_passages(df, global_passage_id, passage_length):
#     """
#     Splits a document into passages of sentences.
    
#     Expects df on format "_id", "concatenated_content"
    
#     Params:
#     - df (DataFrame): DataFrame with the documents concatenated by '_id'
#     - global_passage_id (int): Starting passage ID for the entire data processing
#     - passage_length (int): The number of sentences in each passage

#     Returns:
#     - DataFrame with the columns "_id", "concatenated_content", "Passage_id", "passage"
#     - New global_passage_id after processing the given DataFrame
#     """
#     # Initialize the tokenizer
#     tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
#     passages_data = []

#     # Iterate through each document
#     for index, row in df.iterrows():
#         doc_id = row['_id']
#         content = row['concatenated_content']
#         sentences = tokenizer.tokenize(content)
        
#         for i in range(0, len(sentences), passage_length):
#             passage = ' '.join(sentences[i:i+passage_length])
#             passages_data.append([doc_id, global_passage_id, passage])
#             global_passage_id += 1

#     passages_df = pd.DataFrame(passages_data, columns=['_id', 'Passage_id', 'passage'])
#     return passages_df, global_passage_id

# Latest code with ALL columns

In [74]:
def concatonate_docs(df):
    """
    Concatenates lines with the same _id and preserves specific columns.

    Params:
    - df (DataFrame): The DataFrame to process.

    Returns:
    - DataFrame: DataFrame with _id, concatenated content, 'wellbore name', 
                 'CREATOR BA NAME', and 'year'.
    """
    # Group by _id while concatenating 'content' and keeping the first instance of the other specified columns
    grouped = df.groupby('_id').agg({
        'content': lambda contents: ' '.join(contents.dropna().astype(str)),
        'WELLBORE NAME': 'first',
        'CREATOR BA NAME': 'first',
        'year': 'first'  # assuming 'year' has already been precomputed before calling this function
    }).reset_index()

    # Rename the column to match the expected structure and return the DataFrame
    grouped.rename(columns={'content': 'concatenated_content'}, inplace=True)
    return grouped

def filter_sentences(doc):
    """
    Filters out sentence-like strings such as enumerations or headings which are not actual sentences.

    Params:
    - doc (spaCy Doc): A spaCy Doc object containing tokenized text.

    Returns:
    - List: A filtered list of sentence strings.
    """
    # Prepare an empty list to store the filtered sentences
    sentences = []

    # Define a regex pattern to match headings or enumerations
    pattern = r'\d+(\.\d+)*\s+[A-Z]'

    # Check each sentence and add it to the list if it doesn't match the pattern
    for sent in doc.sents:
        if not re.match(pattern, sent.text):
            sentences.append(sent.text.strip())

    return sentences

def define_passages(df, global_passage_id, passage_length):
    """
    Splits documents into passages of complete sentences based on the specified passage length.
    
    Params:
    - df (DataFrame): DataFrame with documents concatenated by '_id'.
    - global_passage_id (int): An integer ID that uniquely identifies each passage.
    - passage_length (int): The number of sentences in each passage.
    
    Returns:
    - DataFrame: A new DataFrame with columns '_id', 'Passage_id', 'passage'.
    - int: The updated global passage ID after processing the DataFrame.
    """
    # Initialize a list to store passage data
    passages_data = []

    # Your logic here to tokenize and create passages, appending them to passages_data
    # Iterate through each row in the DataFrame
    for index, row in df.iterrows():
        # Extract the document ID and concatenated content for the current row
        doc_id = row['_id']
        content = row['concatenated_content']
        
        # Process the content using spaCy to tokenize into sentences
        doc = nlp(content)
        
        # Filter the sentences using the custom filter function
        sentences = filter_sentences(doc)

        # Group sentences into passages of the specified length
        for i in range(0, len(sentences), passage_length):
            # Join sentences to form a single passage text
            passage = ' '.join(sentences[i:i+passage_length])
            # Append the passage data to the list, including the unique passage ID
            passages_data.append([doc_id, global_passage_id, passage, row['WELLBORE NAME'], row['CREATOR BA NAME'],row['year']])
            # Increment the unique passage ID for the next passage
            global_passage_id += 1



    # After processing all rows and creating passage texts with a unique ID:
    passages_df = pd.DataFrame(passages_data, columns=['_id', 'Passage_id', 'passage','WELLBORE NAME', 'CREATOR BA NAME', 'year'])

    # Return the DataFrame of passages along with the updated global passage ID
    return passages_df, global_passage_id

import pandas as pd
import spacy
import re

# Assuming spacy and nltk punkt tokenizer are already loaded
nlp = spacy.load("en_core_web_sm")

# Assuming the functions filter_sentences and define_passages remain unchanged

def concatonate_docs(df):
    """
    Concatenates 'content' for rows with the same '_id', while also retaining other specified columns.
    Incorporates the 'wellbore name', 'CREATOR BA NAME', and 'year' columns into the final DataFrame.

    Parameters:
    - df: DataFrame containing the document and metadata.

    Returns:
    - DataFrame containing '_id', the concatenated 'content', 'wellbore name', 'CREATOR BA NAME', and 'year'.
    """
    # Group the DataFrame by '_id' and concatenate the content for each group
    # Also, keep the first encountered non-null value for 'wellbore name', 'CREATOR BA NAME', and 'year'
    grouped_content = df.groupby('_id').agg({
        'content': lambda x: ''.join(x.dropna().astype(str)),
        'WELLBORE NAME': 'first',  # Retain 'wellbore name'
        'CREATOR BA NAME': 'first',  # Retain 'CREATOR BA NAME'
        'year': 'first'  # Retain the 'year', extracted from 'RELEASE DATE' beforehand
    }).reset_index()

    # Rename 'content' column to 'concatenated_content'
    grouped_content.rename(columns={'content': 'concatenated_content'}, inplace=True)

    return grouped_content

def base_df(df, passage_length):
    """
    Processes the input DataFrame, filters it by 'INFO ITEM TYPE', and then creates passages 
    from the 'content' column. Also retains 'wellbore name', 'CREATOR BA NAME', and extracts 
    the year from 'RELEASE DATE'.

    Parameters:
    - df: DataFrame containing the original documents and metadata.
    - passage_length: The desired number of sentences in each passage.

    Returns:
    - DataFrame with '_id', 'Passage_id', 'passage', 'wellbore name', 'CREATOR BA NAME', 'year', 
      and 'INFO_ITEM_TYPE'.
    """
    # Create a copy of the filtered DataFrame to avoid 'SettingWithCopyWarning'
    # Extract the year from 'RELEASE DATE'
    df = df.copy()
    df['year'] = pd.to_datetime(df['RELEASE DATE']).dt.year

    # Initialize the global passage ID
    global_passage_id = 0

    # Prepare a list to store the combined results from all unique 'INFO ITEM TYPE' entries
    result_dfs = []

    # Get the unique 'INFO ITEM TYPE' values
    for item_type in df['INFO ITEM TYPE'].unique():
        # Filter the DataFrame by 'INFO ITEM TYPE' and whether the content is natural language in English
        df_filtered = df[(df['INFO ITEM TYPE'] == item_type) &
                         (df['content_could_be_natural_language']) &
                         (df['possible_language'] == 'en')]

        # Concatenate the content for rows with the same '_id' and retain additional specified columns
        concat_df = concatonate_docs(df_filtered)

        # Split the concatenate content into passages
        passage_df, global_passage_id = define_passages(concat_df, global_passage_id, passage_length)
        print(f"Passage_df shape: {passage_df.shape}")

        # Add 'INFO ITEM TYPE' back into the resulting DataFrame
        passage_df['INFO_ITEM_TYPE'] = item_type

        # Append the passages DataFrame to the lis
        # t for concatenation later
        result_dfs.append(passage_df)

    # Concatenate all intermediate passage DataFrames into one
    final_df = pd.concat(result_dfs, ignore_index=True)
    print(final_df.columns)

    # Return the final DataFrame with the specified columns
    return final_df[['INFO_ITEM_TYPE', '_id', 'Passage_id', 'passage', 'WELLBORE NAME', 'CREATOR BA NAME', 'year']]

In [75]:
desired_df = base_df(df[df['INFO ITEM TYPE'] == 'DISCOVERY_EVALUATION_REPORT'],passage_length=50) 

Passage_df shape: (50, 6)
Index(['_id', 'Passage_id', 'passage', 'WELLBORE NAME', 'CREATOR BA NAME',
       'year', 'INFO_ITEM_TYPE'],
      dtype='object')


In [77]:
desired_df.to_csv()

(50, 7)

In [64]:
df

,_id,filename,page,content,possible_language,langdetect,content_could_be_natural_language,content_scrubbed_light,content_scrubbed_light_could_be_natural_language,WELLBORE NAME,...,DATA COLLECTION,CREATION PROCESS,DATA DOMAIN,REMARK,UNLOAD FILE PATH,UNLOAD FILE NAME,INFORMATION ITEM ID,Dataset Id,Last Modified,Update Time
0,997d83feeb81f8044b6447210ab739ad1e9a4f7f,15_6-5__WELL__15-06-05_PB-706-0719.pdf,6,The Upper Dogger The Interval from 3627 to 368...,en,[en:0.9999977940633832],True,The Upper Dogger The Interval from 3627 to 368...,True,15/6-5,...,DISKOS SCANNED COMPLETION REPORTS,INTERP,WL.CMPL,NaN,NaN,NaN,3.182072e+08,14.05.01.00,2022-03-29 00:00:00,2023-08-28 12:22:54
1,997d83feeb81f8044b6447210ab739ad1e9a4f7f,15_6-5__WELL__15-06-05_PB-706-0719.pdf,7,The above is underlain by 272 in thick sandsto...,en,[en:0.9999952423862815],True,The above is underlain by 272 in thick sandsto...,True,15/6-5,...,DISKOS SCANNED COMPLETION REPORTS,INTERP,WL.CMPL,NaN,NaN,NaN,3.182072e+08,14.05.01.00,2022-03-29 00:00:00,2023-08-28 12:22:54
2,997d83feeb81f8044b6447210ab739ad1e9a4f7f,15_6-5__WELL__15-06-05_PB-706-0719.pdf,8,"seen in the main Sleipner reservoir to date, -...",en,[en:0.9999966473882063],True,"seen in the main Sleipner reservoir to date, -...",True,15/6-5,...,DISKOS SCANNED COMPLETION REPORTS,INTERP,WL.CMPL,NaN,NaN,NaN,3.182072e+08,14.05.01.00,2022-03-29 00:00:00,2023-08-28 12:22:54
3,997d83feeb81f8044b6447210ab739ad1e9a4f7f,15_6-5__WELL__15-06-05_PB-706-0719.pdf,9,TABLE WIRELINE LOGGING SUMMARY Well 15/6-5 Log...,en,[en:0.999994410115181],False,TABLE WIRELINE LOGGING SUMMARY Well 15/6-5 Log...,False,15/6-5,...,DISKOS SCANNED COMPLETION REPORTS,INTERP,WL.CMPL,NaN,NaN,NaN,3.182072e+08,14.05.01.00,2022-03-29 00:00:00,2023-08-28 12:22:54
4,997d83feeb81f8044b6447210ab739ad1e9a4f7f,15_6-5__WELL__15-06-05_PB-706-0719.pdf,10,WELLSITE SAMPLE DESCRIPTION HOLE SIZE: GEOL: R...,en,[en:0.9999961362929385],True,WELLSITE SAMPLE DESCRIPTION HOLE SIZE: GEOL: R...,True,15/6-5,...,DISKOS SCANNED COMPLETION REPORTS,INTERP,WL.CMPL,NaN,NaN,NaN,3.182072e+08,14.05.01.00,2022-03-29 00:00:00,2023-08-28 12:22:54
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
252376,d8b5ec0eb354bbf9e4865565faa4a890dbabfb45,15_9-12__ROCK AND CORE__CORE_SCAL_1982-10-30_R...,48,"wycousawNmp,moom-Liwm e8-1301 (opds aJod luaoJ...",en,"[en:0.5714274116776232, pt:0.2857141457066634,...",False,"wycousawNmp,moom-Liwm e8-1301 (opds aJod luaoJ...",False,15/9-12,...,DISKOS LEGACY DATA,LAB,WL.RC.SCAL,NaN,WELL_15_9-12/WB_309-L/CORE_DATA_1982-10-30,CORE_SCAL_1982-10-30_REPORT_1.PDF,1.017913e+10,02.25.01.00,2023-04-25 00:00:00,2023-07-12 07:25:22
252377,d8b5ec0eb354bbf9e4865565faa4a890dbabfb45,15_9-12__ROCK AND CORE__CORE_SCAL_1982-10-30_R...,49,"IrVEN,114111NMPuelgeoW-liO'su af-EM11 (Boods a...",en,"[pt:0.8571396095099522, en:0.1428589548695462]",False,"IrVEN,114111NMPuelgeoW-liO'su af-EM11 (Boods a...",False,15/9-12,...,DISKOS LEGACY DATA,LAB,WL.RC.SCAL,NaN,WELL_15_9-12/WB_309-L/CORE_DATA_1982-10-30,CORE_SCAL_1982-10-30_REPORT_1.PDF,1.017913e+10,02.25.01.00,2023-04-25 00:00:00,2023-07-12 07:25:22
252378,d8b5ec0eb354bbf9e4865565faa4a890dbabfb45,15_9-12__ROCK AND CORE__CORE_SCAL_1982-10-30_R...,50,Pressure (bar II Capillary Pressure Curve Comp...,en,"[en:0.7142842662068939, pt:0.28571429811363974]",False,Pressure (bar II Capillary Pressure Curve Comp...,False,15/9-12,...,DISKOS LEGACY DATA,LAB,WL.RC.SCAL,NaN,WELL_15_9-12/WB_309-L/CORE_DATA_1982-10-30,CORE_SCAL_1982-10-30_REPORT_1.PDF,1.017913e+10,02.25.01.00,2023-04-25 00:00:00,2023-07-12 07:25:22
252379,d8b5ec0eb354bbf9e4865565faa4a890dbabfb45,15_9-12__ROCK AND CORE__CORE_SCAL_1982-10-30_R...,51,Company STATQIL A/S Depth.. S.Q57...05...........,en,"[en:0.571427361092836, pt:0.42857077727642967]",False,Company STATQIL A/S Depth.. S.Q57...05Gr. Dens...,False,15/9-12,...,DISKOS LEGACY DATA,LAB,WL.RC.SCAL,NaN,WELL_15_9-12/WB_309-L/CORE_DATA_1982-10-30,CORE_SCAL_1982-10-30_REPORT_1.PDF,1.017913e+10,02.25